In [95]:
from datasets import load_dataset
from dotenv import load_dotenv
import os
from huggingface_hub import login
import pandas as pd
from generator_flow import generator
from generator_flow import normal_generator

Load examples

In [96]:
load_dotenv()
login(os.getenv("HF_API_KEY"))
dataset = load_dataset("Tamiza/zimpl_data")
df_manual = dataset['test'].to_pandas()
dataset

DatasetDict({
    test: Dataset({
        features: ['input', 'output', 'parameter', 'solution'],
        num_rows: 67
    })
})

Table for manual tests

In [97]:
df_manual


,input,output,parameter,solution
0,"A bakery produces two types of cookies: chocolate chip and caramel. The bakery anticipates daily demand for a minimum of 80 caramelized & 120 chocolate chip cookies. Due to a lack of raw materials and labor, the bakery can produce 120 caramel cookies and 140 chocolate chip cookies daily. For the bakery to be viable, it must sell a minimum of 240 cookies each day. Every chocolate chip cookie served generates $0.75 in profit, whereas each caramel biscuit generates $0.88. The solution to the number of chocolate chip and caramel cookies that the bakery must produce each day to maximize profit may be determined using linear programming.","set CookieTypes := {""Caramel"", ""ChocolateChip""};\n\n# Parameters for daily demand and production limits\nparam min_demand[CookieTypes] :=\n <""Caramel""> 80,\n <""ChocolateChip""> 120;\n\nparam max_production[CookieTypes] :=\n <""Caramel""> 120,\n <""ChocolateChip""> 140;\n\n# Parameter for minimum total sales\nparam min_total_sales := 240;\n\n# Parameters for profit per cookie (in dollars)\nparam profit[CookieTypes] :=\n <""Caramel""> 0.88,\n <""ChocolateChip""> 0.75;\n\n# Decision variables: quantity of each cookie type produced daily\nvar x[<c> in CookieTypes] >= min_demand[c];\n\n# Objective function: Maximize total profit\nmaximize total_profit: sum <c> in CookieTypes: profit[c] * x[c];\n\n# Constraints: Ensure minimum total cookies are sold\nsubto total_sales:\n sum <c> in CookieTypes: x[c] >= min_total_sales;\n\n# Constraints: Do not exceed daily production limits\nsubto production_limits:\n forall <c> in CookieTypes do\n x[c] <= max_production[c];\n",1,objective value: 210.6\nx$Caramel 120 \t(obj:0.88)\nx$ChocolateChip 140 \t(obj:0.75)
1,"""The factory produces two products, W1 and W2, using resources S1, S2, and S3. \n\nTo produce W1, the factory uses 2 units of S1, 1 unit of S2, and 4 units of S3. \n\nTo produce W2, the factory uses 2 units of S1 and 2 units of S2. \n\nThe weekly limit for resources is 14, 8, and 16 units, respectively. \n\nThe selling prices of the products are $2 and $3, respectively. \n\nDetermine the production plan that would maximize the profit from sales.""","set Products := {""W1"", ""W2""};\nset Resources := {""S1"", ""S2"", ""S3""};\n\n# Parameters for resource usage per product\nparam resource_usage[Products * Resources] :=\n | ""S1"", ""S2"", ""S3"" |\n|""W1"" | 2, 1, 4 |\n|""W2"" | 2, 2, 0 |;\n\n# Weekly resource limits\nparam resource_limits[Resources] :=\n <""S1""> 14,\n <""S2""> 8,\n <""S3""> 16;\n\n# Selling price per unit of each product\nparam selling_price[Products] :=\n <""W1""> 2,\n <""W2""> 3;\n\n# Decision variables: weekly production quantity of each product\nvar x[<p> in Products] >= 0;\n\n# Objective function: Maximize total profit\nmaximize total_profit: sum <p> in Products: selling_price[p] * x[p];\n\n# Constraints: Ensure resource usage does not exceed weekly limits\nsubto resource_constraints:\n forall <r> in Resources do\n sum <p> in Products: resource_usage[p, r] * x[p] <= resource_limits[r];",1,objective value: 14\nx$W1 4 \t(obj:2)\nx$W2 2 \t(obj:3)
2,"A mining company produces lignite and anthracite.\nBy the moment, it is able to sell all the coal produced, being the profit per ton of lignite and anthracite 4 and 3 monetary units, respectively. Processing each ton of lignite requires 3 hours of coal cutting machine and another 4 hours for washing.\nAlso, the processing of one ton of anthracite required for same tasks 4 and 2 hours, respectively\nThe time available daily to each of these activities (cutting and washing) is 12 and 8 hours respectively. Furthermore, it is desired to produce daily 3 tons of coal.","set CoalTypes := {""Lignite"", ""Anthracite""};\nset Activities := {""Cutting"", ""Washing""};\n\n# Parameters for time required per ton for each activity\nparam time_per_ton[CoalTypes * Activities] :=\n | ""Cutting"", ""Washing"" |\n|""Lignite"" | 3, 4 |\n|""Anthracite""| 4, 2 |;\n\n# Avai

Generate responses using AI ZIMPL Generator (for now first 10)

In [98]:
df_manual['ai_advanced'] = df_manual.apply(lambda row: generator(row['input'], row['parameter']), axis=1)

Show answers

In [99]:
df_manual[:5].map(lambda x: str(x)).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

,input,output,parameter,solution,ai_advanced
0,"A bakery produces two types of cookies: chocolate chip and caramel. The bakery anticipates daily demand for a minimum of 80 caramelized & 120 chocolate chip cookies. Due to a lack of raw materials and labor, the bakery can produce 120 caramel cookies and 140 chocolate chip cookies daily. For the bakery to be viable, it must sell a minimum of 240 cookies each day. Every chocolate chip cookie served generates $0.75 in profit, whereas each caramel biscuit generates $0.88. The solution to the number of chocolate chip and caramel cookies that the bakery must produce each day to maximize profit may be determined using linear programming.","set CookieTypes := {""Caramel"", ""ChocolateChip""}; # Parameters for daily demand and production limits param min_demand[CookieTypes] := <""Caramel""> 80, <""ChocolateChip""> 120; param max_production[CookieTypes] := <""Caramel""> 120, <""ChocolateChip""> 140; # Parameter for minimum total sales param min_total_sales := 240; # Parameters for profit per cookie (in dollars) param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables: quantity of each cookie type produced daily var x[ in CookieTypes] >= min_demand[c]; # Objective function: Maximize total profit maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints: Ensure minimum total cookies are sold subto total_sales: sum in CookieTypes: x[c] >= min_total_sales; # Constraints: Do not exceed daily production limits subto production_limits: forall in CookieTypes do x[c] <= max_production[c];",1,objective value: 210.6 x$Caramel 120 (obj:0.88) x$ChocolateChip 140 (obj:0.75),"# Sets set Cookies := {""ChocolateChip"", ""Caramel""}; set Demand := {""MinDemand"", ""MaxProduction""}; set Profits := {""ChocolateChipProfit"", ""CaramelProfit""}; # Parameters for daily demand param demand[Demand] := <""MinDemand""> 240; # Parameters for maximum production limits param max_production[Cookies] := <""ChocolateChip""> 140, <""Caramel""> 120; # Parameters for profits per cookie param profit[Cookies] := <""ChocolateChip""> 0.75, <""Caramel""> 0.88; # Parameters for minimum demand per cookie type param min_demand[Cookies] := <""ChocolateChip""> 120, <""Caramel""> 80; # Decision variables var x[ in Cookies] integer >= min_demand[c] <= max_production[c]; # Number of cookies produced for each type # Objective function maximize total_profit: sum in Cookies: profit[c] * x[c]; # Constraints subto demand_constraint: sum in Cookies: x[c] >= demand[""MinDemand""]; # Ensure minimum total cookie production subto min_demand_constraints: forall in Cookies: x[c] >= min_demand[c]; # Ensure minimum demand for each cookie type subto max_production_constraints: forall in Cookies: x[c] <= max_production[c]; # Ensure maximum production limits for each cookie type"
1,"""The factory produces two products, W1 and W2, using resources S1, S2, and S3. To produce W1, the factory uses 2 units of S1, 1 unit of S2, and 4 units of S3. To produce W2, the factory uses 2 units of S1 and 2 units of S2. The weekly limit for resources is 14, 8, and 16 units, respectively. The selling prices of the products are $2 and $3, respectively. Determine the production plan that would maximize the profit from sales.""","set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; # Parameters for resource usage per product param resource_usage[Products * Resources] := | ""S1"", ""S2"", ""S3"" | |""W1"" | 2, 1, 4 | |""W2"" | 2, 2, 0 |; # Weekly resource limits param resource_limits[Resources] := <""S1""> 14, <""S2""> 8, <""S3""> 16; # Selling price per unit of each product param selling_price[Products] := <""W1""> 2, <""W2""> 3; # Decision variables: weekly production quantity of each product var x[ in Products] >= 0; # Objective function: Maximize total profit maximize total_profit: sum in Products: selling_price[p] * x[p]; # Constraints: Ensure resource usage does not exceed weekly limits su

In [100]:
df_manual['ai_normal'] = df_manual['input'].apply(normal_generator)

In [101]:
from pandas import DataFrame
df_manual[:5][['output', 'ai_advanced', 'ai_normal']].map(lambda x: str(x)).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

,output,ai_advanced,ai_normal
0,"set CookieTypes := {""Caramel"", ""ChocolateChip""}; # Parameters for daily demand and production limits param min_demand[CookieTypes] := <""Caramel""> 80, <""ChocolateChip""> 120; param max_production[CookieTypes] := <""Caramel""> 120, <""ChocolateChip""> 140; # Parameter for minimum total sales param min_total_sales := 240; # Parameters for profit per cookie (in dollars) param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables: quantity of each cookie type produced daily var x[ in CookieTypes] >= min_demand[c]; # Objective function: Maximize total profit maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints: Ensure minimum total cookies are sold subto total_sales: sum in CookieTypes: x[c] >= min_total_sales; # Constraints: Do not exceed daily production limits subto production_limits: forall in CookieTypes do x[c] <= max_production[c];","# Sets set Cookies := {""ChocolateChip"", ""Caramel""}; set Demand := {""MinDemand"", ""MaxProduction""}; set Profits := {""ChocolateChipProfit"", ""CaramelProfit""}; # Parameters for daily demand param demand[Demand] := <""MinDemand""> 240; # Parameters for maximum production limits param max_production[Cookies] := <""ChocolateChip""> 140, <""Caramel""> 120; # Parameters for profits per cookie param profit[Cookies] := <""ChocolateChip""> 0.75, <""Caramel""> 0.88; # Parameters for minimum demand per cookie type param min_demand[Cookies] := <""ChocolateChip""> 120, <""Caramel""> 80; # Decision variables var x[ in Cookies] integer >= min_demand[c] <= max_production[c]; # Number of cookies produced for each type # Objective function maximize total_profit: sum in Cookies: profit[c] * x[c]; # Constraints subto demand_constraint: sum in Cookies: x[c] >= demand[""MinDemand""]; # Ensure minimum total cookie production subto min_demand_constraints: forall in Cookies: x[c] >= min_demand[c]; # Ensure minimum demand for each cookie type subto max_production_constraints: forall in Cookies: x[c] <= max_production[c]; # Ensure maximum production limits for each cookie type","var c: >= 0; // number of chocolate chip cookies produced var k: >= 0; // number of caramel cookies produced // Objective: Maximize profit maximize profit: 0.75 * c + 0.88 * k; // Constraints s.t. demand_chocolate: c >= 120; // minimum demand for chocolate chip cookies s.t. demand_caramel: k >= 80; // minimum demand for caramel cookies s.t. production_chocolate: c <= 140; // maximum production capacity for chocolate chip cookies s.t. production_caramel: k <= 120; // maximum production capacity for caramel cookies s.t. total_production: c + k >= 240; // minimum total cookies produced // Solve the model solve; // Output the results display c, k, profit;"
1,"set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; # Parameters for resource usage per product param resource_usage[Products * Resources] := | ""S1"", ""S2"", ""S3"" | |""W1"" | 2, 1, 4 | |""W2"" | 2, 2, 0 |; # Weekly resource limits param resource_limits[Resources] := <""S1""> 14, <""S2""> 8, <""S3""> 16; # Selling price per unit of each product param selling_price[Products] := <""W1""> 2, <""W2""> 3; # Decision variables: weekly production quantity of each product var x[ in Products] >= 0; # Objective function: Maximize total profit maximize total_profit: sum in Products: selling_price[p] * x[p]; # Constraints: Ensure resource usage does not exceed weekly limits subto resource_constraints: forall in Resources do sum in Products: resource_usage[p, r] * x[p] <= resource_limits[r];","# Sets set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; # Decision variables for production quantities var produce[Products] >= 0; # Decision variables var produce[ in Products] integer >= 0; # Quantity of each product produced # Objective function maximize profit: sum in Products: prices[p] * produce[p]; # Constraints subto resource_limits: forall in Resources 

## Solver results

In [102]:
from solver import run_scip_and_extract_solution
SCIP_PATH = r"C:\\Program Files\\SCIPOptSuite 9.1.0\\bin"

df_manual["ai_advanced"] = df_manual["ai_advanced"].astype(str)
df_manual["ai_normal"] = df_manual["ai_normal"].astype(str)

def process_zimpl_code(zimpl_code):
    try:
        valid, result = run_scip_and_extract_solution(zimpl_code, SCIP_PATH)
        print(f"Valid: {valid}, Result: {result}")
        return valid, result
    except Exception as e:
        print(f"Error processing code: {e}")
        return False, None



df_manual[["valid_advanced", "result_advanced"]] = df_manual["ai_advanced"].apply(
    lambda code: pd.Series(process_zimpl_code(code))
)

df_manual[["valid_normal", "result_normal"]] = df_manual["ai_normal"].apply(
    lambda code: pd.Series(process_zimpl_code(code))
)

Valid: True, Result: objective value:                                210.6
x$ChocolateChip                                   140 	(obj:0.75)
x$Caramel                                         120 	(obj:0.88)
Valid: False, Result: No solution found
Valid: True, Result: objective value:                                   10
x$Lignite                                           1 	(obj:4)
x$Anthracite                                        2 	(obj:3)
Valid: False, Result: No solution found
Valid: True, Result: objective value:                                   12
x$Food1                                             6 	(obj:2)
Valid: True, Result: objective value:                                 5305
x$ProductA                                        124 	(obj:20)
x$ProductB                                        113 	(obj:25)
Valid: True, Result: objective value:                                 5800
x$Oil1                                            600 	(obj:5)
x$Oil2                           

## Compare validation

In [103]:
import pandas as pd

comparison_table = pd.DataFrame({
    "Category": [
        "Valid (True)",
        "Not Valid (False)",
    ],
    "Count": [
        (df_manual["valid_advanced"] == True).sum(),
        (df_manual["valid_advanced"] == False).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

comparison_table = pd.DataFrame({
    "Category": [
        "Valid hardcoded (True)",
        "Not Valid hardcoded (False)",
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["parameter"] == 0)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["parameter"] == 0)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

comparison_table = pd.DataFrame({
    "Category": [
        "Valid with params (True)",
        "Not Valid with params (False)"
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["parameter"] == 1)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["parameter"] == 1)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

comparison_table_normal = pd.DataFrame({
    "Category": [
        "Valid (True)",
        "Not Valid (False)",
    ],
    "Count": [
        (df_manual["valid_normal"] == True).sum(),
        (df_manual["valid_normal"] == False).sum(),
    ]
})

comparison_table_normal["Percentage"] = (
    comparison_table_normal["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table_normal)

comparison_table = pd.DataFrame({
    "Category": [
        "Both Valid (True, True)",
        "Advanced Valid Only (True, False)",
        "Normal Valid Only (False, True)",
        "Both Invalid (False, False)",
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["valid_normal"] == True)).sum(),
        ((df_manual["valid_advanced"] == True) & (df_manual["valid_normal"] == False)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["valid_normal"] == True)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["valid_normal"] == False)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)


            Category  Count  Percentage
0       Valid (True)     46       68.66
1  Not Valid (False)     21       31.34
                      Category  Count  Percentage
0       Valid hardcoded (True)     28       41.79
1  Not Valid hardcoded (False)      6        8.96
                        Category  Count  Percentage
0       Valid with params (True)     18       26.87
1  Not Valid with params (False)     15       22.39
            Category  Count  Percentage
0       Valid (True)      0         0.0
1  Not Valid (False)     67       100.0
                            Category  Count  Percentage
0            Both Valid (True, True)      0        0.00
1  Advanced Valid Only (True, False)     46       68.66
2    Normal Valid Only (False, True)      0        0.00
3        Both Invalid (False, False)     21       31.34


In [104]:
from IPython.display import display, HTML

df_manual["ai_advanced_display"] = df_manual["ai_advanced"].str.replace('\n', '<br>', regex=False)

html_content = df_manual["ai_advanced_display"].apply(lambda x: f'<div>{x}</div>').to_list()
display(HTML(''.join(html_content)))

In [105]:
from obj_fun_validator import compare_solutions



TODO

- konfiguracja solvera z notatnikiem i sprawdzanie wyników generowanego kodu

- weryfikacja kodu poprzez: sprawdzanie czy działa, porównywanie wyników, weryfikacja kodu i punktowanie przez niezależy model językowy (kolumna ai_advanced)

- weryfikacja odpowiedzi standardowego gtp 4o-mini (kolumna ai_normal)

- porówanie stworzonego generatora z odpowiedziami standardowego gpt 4o-mini